In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install magenta==2.1.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.4 MB 30.8 MB/s 
     |████████████████████████████████| 1.4 MB 31.9 MB/s 
     |████████████████████████████████| 352 kB 56.0 MB/s 
     |████████████████████████████████| 5.6 MB 30.7 MB/s 
     |████████████████████████████████| 254 kB 44.0 MB/s 
     |████████████████████████████████| 14.5 MB 38.2 MB/s 
     |████████████████████████████████| 69 kB 8.2 MB/s 
     |████████████████████████████████| 90 kB 10.2 MB/s 
     |████████████████████████████████| 204 kB 49.5 MB/s 
     |████████████████████████████████| 2.3 MB 53.6 MB/s 
     |████████████████████████████████| 209 kB 68.8 MB/s 
     |████████████████████████████████| 3.6 MB 52.5 MB/s 
     |████████████████████████████████| 526 kB 62.5 MB/s 
     |████████████████████████████████| 2.5 MB 46.5 MB/s 
     |████████████████████████████████| 151 kB 64.6 MB/s 
     |███████████████████████████

In [3]:
import numpy as np
import pathlib
import zipfile
import os
import pandas as pd
import IPython
import collections
import note_seq
import hashlib

from note_seq import abc_parser
from note_seq import midi_io
from note_seq import musicxml_reader

from magenta.common import merge_hparams
from magenta.contrib import training as contrib_training
from magenta.models.music_vae import MusicVAE
from magenta.models.music_vae import lstm_models
from magenta.models.music_vae import data
from magenta.scripts import convert_dir_to_note_sequences
from magenta.scripts.convert_dir_to_note_sequences import convert_directory
from magenta.models.music_vae import configs
from magenta.models.music_vae.trained_model import TrainedModel
import tensorflow.compat.v1 as tf
import tf_slim

/usr/local/lib/python3.8/dist-packages/resampy/interpn.py:114: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  _resample_loop_p(x, t_out, interp_win, interp_delta, num_table, scale, y)


In [4]:
path = '/content/drive/MyDrive/Colab Notebooks/MusicVAE'
os.mkdir(path + '/data_dir')

In [ ]:
# Load the full GMD with MIDI only (no audio) as a tf.data.Dataset
# dataset = tfds.load(name='groove/4bar-midionly', split=tfds.Split.TRAIN, try_gcs=True)

In [5]:
url = "https://storage.googleapis.com/magentadata/datasets/groove/groove-v1.0.0-midionly.zip"
dir = tf.keras.utils.get_file(origin=url, fname= path + '/data_dir/data.zip', extract=True)
data_dir = pathlib.Path(dir)

3260318/3260318 [==============================] - 0s 0us/step


In [7]:
zipfile.ZipFile(path + '/data_dir/data.zip').extractall(path)

In [8]:
data_root= path + '/groove'
tfrec_root = path + '/data_dir/music.tfrecord'

In [9]:
##### ?converted midi file check...
convert_directory(data_root,tfrec_root,recursive=True)

In [11]:
# configs.py

class Config(collections.namedtuple(
    'Config',
    ['model', 'hparams', 'note_sequence_augmenter', 'data_converter',
     'train_examples_path', 'eval_examples_path', 'tfds_name'])):

    def values(self):
        return self._asdict()

Config.__new__.__defaults__ = (None,) * len(Config._fields)

def update_config(config, update_dict):
    config_dict = config.values()
    config_dict.update(update_dict)
    return Config(**config_dict)

CONFIG_MAP = {}

HParams = contrib_training.HParams

CONFIG_MAP['groovae_4bar'] = Config(
    model=MusicVAE(lstm_models.BidirectionalLstmEncoder(),
                   lstm_models.GrooveLstmDecoder()),
    hparams=merge_hparams(
        lstm_models.get_default_hparams(),
        HParams(
            batch_size=512,
            max_seq_len=16 * 4,
            z_size=256,
            enc_rnn_size=[512],
            dec_rnn_size=[256, 256],
            max_beta=0.2,
            free_bits=48,
            dropout_keep_prob=0.3,
        )),
    note_sequence_augmenter=None,
    data_converter=data.GrooveConverter(
        split_bars=4, steps_per_quarter=4, quarters_per_bar=4,
        max_tensors_per_notesequence=20,
        pitch_classes=data.ROLAND_DRUM_PITCH_CLASSES,

        inference_pitch_classes=data.REDUCED_DRUM_PITCH_CLASSES),
    # tfds_name='groove/4bar-midionly',
    train_examples_path= '/content/data_dir/music.tfrecord')

In [12]:
# music_vae_train.py

def _trial_summary(hparams, examples_path, output_dir):
  """Writes a tensorboard text summary of the trial."""

  examples_path_summary = tf.summary.text(
      'examples_path', tf.constant(examples_path, name='examples_path'),
      collections=[])

  hparams_dict = hparams.values()

  # Create a markdown table from hparams.
  header = '| Key | Value |\n| :--- | :--- |\n'
  keys = sorted(hparams_dict.keys())
  lines = ['| %s | %s |' % (key, str(hparams_dict[key])) for key in keys]
  hparams_table = header + '\n'.join(lines) + '\n'

  hparam_summary = tf.summary.text(
      'hparams', tf.constant(hparams_table, name='hparams'), collections=[])

  with tf.Session() as sess:
    writer = tf.summary.FileWriter(output_dir, graph=sess.graph)
    writer.add_summary(examples_path_summary.eval())
    writer.add_summary(hparam_summary.eval())
    writer.close()


def _get_input_tensors(dataset, config):
  """Get input tensors from dataset."""
  batch_size = config.hparams.batch_size
  iterator = tf.data.make_one_shot_iterator(dataset)
  (input_sequence, output_sequence, control_sequence,
   sequence_length) = iterator.get_next()
  input_sequence.set_shape(
      [batch_size, None, config.data_converter.input_depth])
  output_sequence.set_shape(
      [batch_size, None, config.data_converter.output_depth])
  
  if not config.data_converter.control_depth:
    control_sequence = None

  else:
    control_sequence.set_shape(
        [batch_size, None, config.data_converter.control_depth])
    sequence_length.set_shape([batch_size] + sequence_length.shape[1:].as_list())

  return {
      'input_sequence': input_sequence,
      'output_sequence': output_sequence,
      'control_sequence': control_sequence,
      'sequence_length': sequence_length
  }

def train(train_dir,
          config,
          dataset_fn,
          checkpoints_to_keep=5,
          keep_checkpoint_every_n_hours=1,
          num_steps=None,
          master='',
          num_sync_workers=0,
          num_ps_tasks=0,
          task=0):
    
    # global train_op

#==== train loop ====
    tf.gfile.MakeDirs(train_dir)
    is_chief = (task == 0)
    
    with tf.Graph().as_default():
        with tf.device(tf.train.replica_device_setter(
            num_ps_tasks, merge_devices=True)):
            
            model = config.model
            model.build(config.hparams,
                        config.data_converter.output_depth,
                        is_training=True)
            optimizer = model.train(**_get_input_tensors(dataset_fn(), config))

            hooks = []
            if num_sync_workers:
                optimizer = tf.train.SyncReplicasOptimizer(
                    optimizer,num_sync_workers)
                hooks.append(optimizer.make_session_run_hook(is_chief))

            grads, var_list = list(zip(*optimizer.compute_gradients(model.loss)))
            global_norm = tf.global_norm(grads)
            tf.summary.scalar('global_norm', global_norm)
            
            if config.hparams.clip_mode == 'value':
                g = config.hparams.grad_clip
                clipped_grads = [tf.clip_by_value(grad, -g, g) for grad in grads]
            elif config.hparams.clip_mode == 'global_norm':
                clipped_grads = tf.cond(
                    global_norm < config.hparams.grad_norm_clip_to_zero,
                    lambda: tf.clip_by_global_norm(  # pylint:disable=g-long-lambda
                        grads, config.hparams.grad_clip, use_norm=global_norm)[0],
                    lambda: [tf.zeros(tf.shape(g)) for g in grads])
            else:
                raise ValueError(
                    'Unknown clip_mode: {}'.format(config.hparams.clip_mode))
                train_op = optimizer.apply_gradients(
                    list(zip(clipped_grads, var_list)),
                    global_step=model.global_step,
                    name='train_step')
            logging_dict = {'global_step': model.global_step,
                            'loss': model.loss}
            
            hooks.append(tf.train.LoggingTensorHook(logging_dict, every_n_iter=100))
            if num_steps:
                hooks.append(tf.train.StopAtStepHook(last_step=num_steps))
                
            scaffold = tf.train.Scaffold(
                saver=tf.train.Saver(
                    max_to_keep=checkpoints_to_keep,
                    keep_checkpoint_every_n_hours=keep_checkpoint_every_n_hours))
            
            tf_slim.training.train(
                train_op=train_op,
                logdir=train_dir,
                scaffold=scaffold,
                hooks=hooks,
                save_checkpoint_secs=60,
                master=master,
                is_chief=is_chief)

def run(config_map,
        tf_file_reader=tf.data.TFRecordDataset,
        file_reader=tf.python_io.tf_record_iterator,
        is_training=True):
    config = config_map['groovae_4bar']
    train_dir = '/content/train'
    num_steps = 5000
    
    def dataset_fn():
        return data.get_dataset(
            config,
            tf_file_reader=tf_file_reader,
            is_training=True,
            cache_dataset=True)
    
    if is_training == True:
        train(
            train_dir,
            config=config,
            dataset_fn=dataset_fn,
            num_steps=num_steps)      
    
    else:
        print("EVAL")

In [13]:
##### ?error checking...
run(CONFIG_MAP)

ValueError: ignored

In [ ]:
from magenta.models.music_vae import music_vae_train

In [ ]:
drum_config = configs.CONFIG_MAP['groovae_4bar']
model = TrainedModel(drum_config, batch_size=2, checkpoint_dir_or_path= path +'/train')

generated_sequence = model.sample(n=1, length=64, temperature=0.5)
note_seq.sequence_proto_to_midi_file(generated_sequence[0], path + '/gen_midi/drum_4bar_new.mid')

In [ ]:
# """MusicVAE training script."""
# import os

# from magenta.models.music_vae import configs
# from magenta.models.music_vae import data
# import tensorflow.compat.v1 as tf
# import tf_slim

# flags = tf.app.flags
# FLAGS = flags.FLAGS

# flags.DEFINE_string(
#     'master', '',
#     'The TensorFlow master to use.')
# flags.DEFINE_string(
#     'examples_path', None,
#     'Path to a TFRecord file of NoteSequence examples. Overrides the config.')
# flags.DEFINE_string(
#     'tfds_name', None,
#     'TensorFlow Datasets dataset name to use. Overrides the config.')
# flags.DEFINE_string(
#     'run_dir', None,
#     'Path where checkpoints and summary events will be located during '
#     'training and evaluation. Separate subdirectories `train` and `eval` '
#     'will be created within this directory.')
# flags.DEFINE_integer(
#     'num_steps', 200000,
#     'Number of training steps or `None` for infinite.')
# flags.DEFINE_integer(
#     'eval_num_batches', None,
#     'Number of batches to use during evaluation or `None` for all batches '
#     'in the data source.')
# flags.DEFINE_integer(
#     'checkpoints_to_keep', 100,
#     'Maximum number of checkpoints to keep in `train` mode or 0 for infinite.')
# flags.DEFINE_integer(
#     'keep_checkpoint_every_n_hours', 1,
#     'In addition to checkpoints_to_keep, keep a checkpoint every N hours.')
# flags.DEFINE_string(
#     'mode', 'train',
#     'Which mode to use (`train` or `eval`).')
# flags.DEFINE_string(
#     'config', '',
#     'The name of the config to use.')
# flags.DEFINE_string(
#     'hparams', '',
#     'A comma-separated list of `name=value` hyperparameter values to merge '
#     'with those in the config.')
# flags.DEFINE_bool(
#     'cache_dataset', True,
#     'Whether to cache the dataset in memory for improved training speed. May '
#     'cause memory errors for very large datasets.')
# flags.DEFINE_integer(
#     'task', 0,
#     'The task number for this worker.')
# flags.DEFINE_integer(
#     'num_ps_tasks', 0,
#     'The number of parameter server tasks.')
# flags.DEFINE_integer(
#     'num_sync_workers', 0,
#     'The number of synchronized workers.')
# flags.DEFINE_string(
#     'eval_dir_suffix', '',
#     'Suffix to add to eval output directory.')
# flags.DEFINE_string(
#     'log', 'INFO',
#     'The threshold for what messages will be logged: '
#     'DEBUG, INFO, WARN, ERROR, or FATAL.')


# # Should not be called from within the graph to avoid redundant summaries.
# def _trial_summary(hparams, examples_path, output_dir):
#   """Writes a tensorboard text summary of the trial."""

#   examples_path_summary = tf.summary.text(
#       'examples_path', tf.constant(examples_path, name='examples_path'),
#       collections=[])

#   hparams_dict = hparams.values()

#   # Create a markdown table from hparams.
#   header = '| Key | Value |\n| :--- | :--- |\n'
#   keys = sorted(hparams_dict.keys())
#   lines = ['| %s | %s |' % (key, str(hparams_dict[key])) for key in keys]
#   hparams_table = header + '\n'.join(lines) + '\n'

#   hparam_summary = tf.summary.text(
#       'hparams', tf.constant(hparams_table, name='hparams'), collections=[])

#   with tf.Session() as sess:
#     writer = tf.summary.FileWriter(output_dir, graph=sess.graph)
#     writer.add_summary(examples_path_summary.eval())
#     writer.add_summary(hparam_summary.eval())
#     writer.close()


# def _get_input_tensors(dataset, config):
#   """Get input tensors from dataset."""
#   batch_size = config.hparams.batch_size
#   iterator = tf.data.make_one_shot_iterator(dataset)
#   (input_sequence, output_sequence, control_sequence,
#    sequence_length) = iterator.get_next()
#   input_sequence.set_shape(
#       [batch_size, None, config.data_converter.input_depth])
#   output_sequence.set_shape(
#       [batch_size, None, config.data_converter.output_depth])
#   if not config.data_converter.control_depth:
#     control_sequence = None
#   else:
#     control_sequence.set_shape(
#         [batch_size, None, config.data_converter.control_depth])
#   sequence_length.set_shape([batch_size] + sequence_length.shape[1:].as_list())

#   return {
#       'input_sequence': input_sequence,
#       'output_sequence': output_sequence,
#       'control_sequence': control_sequence,
#       'sequence_length': sequence_length
#   }


# def train(train_dir,
#           config,
#           dataset_fn,
#           checkpoints_to_keep=5,
#           keep_checkpoint_every_n_hours=1,
#           num_steps=None,
#           master='',
#           num_sync_workers=0,
#           num_ps_tasks=0,
#           task=0):
#   """Train loop."""
#   tf.gfile.MakeDirs(train_dir)
#   is_chief = (task == 0)
#   if is_chief:
#     _trial_summary(
#         config.hparams, config.train_examples_path or config.tfds_name,
#         train_dir)
#   with tf.Graph().as_default():
#     with tf.device(tf.train.replica_device_setter(
#         num_ps_tasks, merge_devices=True)):

#       model = config.model
#       model.build(config.hparams,
#                   config.data_converter.output_depth,
#                   is_training=True)

#       optimizer = model.train(**_get_input_tensors(dataset_fn(), config))

#       hooks = []
#       if num_sync_workers:
#         optimizer = tf.train.SyncReplicasOptimizer(
#             optimizer,
#             num_sync_workers)
#         hooks.append(optimizer.make_session_run_hook(is_chief))

#       grads, var_list = list(zip(*optimizer.compute_gradients(model.loss)))
#       global_norm = tf.global_norm(grads)
#       tf.summary.scalar('global_norm', global_norm)

#       if config.hparams.clip_mode == 'value':
#         g = config.hparams.grad_clip
#         clipped_grads = [tf.clip_by_value(grad, -g, g) for grad in grads]
#       elif config.hparams.clip_mode == 'global_norm':
#         clipped_grads = tf.cond(
#             global_norm < config.hparams.grad_norm_clip_to_zero,
#             lambda: tf.clip_by_global_norm(  # pylint:disable=g-long-lambda
#                 grads, config.hparams.grad_clip, use_norm=global_norm)[0],
#             lambda: [tf.zeros(tf.shape(g)) for g in grads])
#       else:
#         raise ValueError(
#             'Unknown clip_mode: {}'.format(config.hparams.clip_mode))
#       train_op = optimizer.apply_gradients(
#           list(zip(clipped_grads, var_list)),
#           global_step=model.global_step,
#           name='train_step')

#       logging_dict = {'global_step': model.global_step,
#                       'loss': model.loss}

#       hooks.append(tf.train.LoggingTensorHook(logging_dict, every_n_iter=100))
#       if num_steps:
#         hooks.append(tf.train.StopAtStepHook(last_step=num_steps))

#       scaffold = tf.train.Scaffold(
#           saver=tf.train.Saver(
#               max_to_keep=checkpoints_to_keep,
#               keep_checkpoint_every_n_hours=keep_checkpoint_every_n_hours))
#       tf_slim.training.train(
#           train_op=train_op,
#           logdir=train_dir,
#           scaffold=scaffold,
#           hooks=hooks,
#           save_checkpoint_secs=60,
#           master=master,
#           is_chief=is_chief)


# def evaluate(train_dir,
#              eval_dir,
#              config,
#              dataset_fn,
#              num_batches,
#              master=''):
#   """Evaluate the model repeatedly."""
#   tf.gfile.MakeDirs(eval_dir)

#   _trial_summary(
#       config.hparams, config.eval_examples_path or config.tfds_name, eval_dir)
#   with tf.Graph().as_default():
#     model = config.model
#     model.build(config.hparams,
#                 config.data_converter.output_depth,
#                 is_training=False)

#     eval_op = model.eval(
#         **_get_input_tensors(dataset_fn().take(num_batches), config))

#     hooks = [
#         tf_slim.evaluation.StopAfterNEvalsHook(num_batches),
#         tf_slim.evaluation.SummaryAtEndHook(eval_dir)
#     ]
#     tf_slim.evaluation.evaluate_repeatedly(
#         train_dir,
#         eval_ops=eval_op,
#         hooks=hooks,
#         eval_interval_secs=60,
#         master=master)


# def run(config_map,
#         tf_file_reader=tf.data.TFRecordDataset,
#         file_reader=tf.python_io.tf_record_iterator):
#   """Load model params, save config file and start trainer.
#   Args:
#     config_map: Dictionary mapping configuration name to Config object.
#     tf_file_reader: The tf.data.Dataset class to use for reading files.
#     file_reader: The Python reader to use for reading files.
#   Raises:
#     ValueError: if required flags are missing or invalid.
#   """
#   if not FLAGS.run_dir:
#     raise ValueError('Invalid run directory: %s' % FLAGS.run_dir)
#   run_dir = os.path.expanduser(FLAGS.run_dir)
#   train_dir = os.path.join(run_dir, 'train')

#   if FLAGS.mode not in ['train', 'eval']:
#     raise ValueError('Invalid mode: %s' % FLAGS.mode)

#   if FLAGS.config not in config_map:
#     raise ValueError('Invalid config: %s' % FLAGS.config)
#   config = config_map[FLAGS.config]
#   if FLAGS.hparams:
#     config.hparams.parse(FLAGS.hparams)
#   config_update_map = {}
#   if FLAGS.examples_path:
#     config_update_map['%s_examples_path' % FLAGS.mode] = os.path.expanduser(
#         FLAGS.examples_path)
#   if FLAGS.tfds_name:
#     if FLAGS.examples_path:
#       raise ValueError(
#           'At most one of --examples_path and --tfds_name can be set.')
#     config_update_map['tfds_name'] = FLAGS.tfds_name
#     config_update_map['eval_examples_path'] = None
#     config_update_map['train_examples_path'] = None
#   config = configs.update_config(config, config_update_map)
#   if FLAGS.num_sync_workers:
#     config.hparams.batch_size //= FLAGS.num_sync_workers

#   if FLAGS.mode == 'train':
#     is_training = True
#   elif FLAGS.mode == 'eval':
#     is_training = False
#   else:
#     raise ValueError('Invalid mode: {}'.format(FLAGS.mode))

#   def dataset_fn():
#     return data.get_dataset(
#         config,
#         tf_file_reader=tf_file_reader,
#         is_training=is_training,
#         cache_dataset=FLAGS.cache_dataset)

#   if is_training:
#     train(
#         train_dir,
#         config=config,
#         dataset_fn=dataset_fn,
#         checkpoints_to_keep=FLAGS.checkpoints_to_keep,
#         keep_checkpoint_every_n_hours=FLAGS.keep_checkpoint_every_n_hours,
#         num_steps=FLAGS.num_steps,
#         master=FLAGS.master,
#         num_sync_workers=FLAGS.num_sync_workers,
#         num_ps_tasks=FLAGS.num_ps_tasks,
#         task=FLAGS.task)
#   else:
#     num_batches = FLAGS.eval_num_batches or data.count_examples(
#         config.eval_examples_path,
#         config.tfds_name,
#         config.data_converter,
#         file_reader) // config.hparams.batch_size
#     eval_dir = os.path.join(run_dir, 'eval' + FLAGS.eval_dir_suffix)
#     evaluate(
#         train_dir,
#         eval_dir,
#         config=config,
#         dataset_fn=dataset_fn,
#         num_batches=num_batches,
#         master=FLAGS.master)


# def main(unused_argv):
#   tf.logging.set_verbosity(FLAGS.log)
#   run(configs.CONFIG_MAP)


# def console_entry_point():
#   tf.disable_v2_behavior()
#   tf.app.run(main)


# if __name__ == '__main__':
#   console_entry_point()